In [1]:
import pandas as pd
# df = pd.read_csv('merged.csv')
# # 过滤predTag列值为1的记录
# filtered_df = df[df['predTag'] == 1]

# # 保存到新的CSV文件
# filtered_df.to_csv('filtered_file.csv', index=False)

In [2]:
gt_df = pd.read_csv('gt.csv')

# 创建一个空的列表用于存储处理后的行
processed_rows = []

# 遍历每个ImageID
for image_id in gt_df['ImageID'].unique():
    subset = gt_df[gt_df['ImageID'] == image_id]

    if len(subset) == 1:
        # 如果ImageID是单值，则不做变动
        processed_rows.append(subset.iloc[0])
    else:
        # 如果ImageID是多值
        if all(subset['labelID'] == 9):
            # 如果所有LabelID均为9，则合并并且单独记为9
            combined_row = subset.iloc[0].copy()
            combined_row['labelID'] = 9
            processed_rows.append(combined_row)
        else:
            # 如果不是所有LabelID均为9，则去掉为9的行，剩下的不做变动
            non_9_rows = subset[subset['labelID'] != 9]
            for _, row in non_9_rows.iterrows():
                processed_rows.append(row)

# 将处理后的行转换为DataFrame
processed_gt_df = pd.DataFrame(processed_rows)

processed_gt_df.to_csv('processed_gt.csv', index=False)

In [3]:
gt_df = pd.read_csv('processed_gt.csv')
filtered_df = pd.read_csv('filtered_predTag_1.csv')

# 初始化一个列表来存储合并后的结果
result_list = []

# 遍历所有ImageID
for image_id in set(gt_df['ImageID']).union(filtered_df['ImageID']):
    gt_labels = gt_df[gt_df['ImageID'] == image_id]['labelID'].tolist()
    filtered_labels = filtered_df[filtered_df['ImageID'] == image_id]['labelID'].tolist()

    gt_all_9 = all(label == 9 for label in gt_labels)
    filtered_all_9 = all(label == 9 for label in filtered_labels)

    if gt_all_9 and filtered_all_9:
        result = 'same pass'
    elif gt_all_9 and not filtered_all_9:
        result = 'overkill'
    elif not gt_all_9 and filtered_all_9:
        result = 'miss kill'
    elif set(gt_labels).isdisjoint(set(filtered_labels)):
        result = 'wrong kill'
    elif any(label != 9 and label in filtered_labels for label in gt_labels):
        result = 'same NG'
    else:
        result = 'same NG'

    # 获取额外的列数据
    gt_data = gt_df[gt_df['ImageID'] == image_id]
    filtered_data = filtered_df[filtered_df['ImageID'] == image_id]

    if len(gt_data) == 0 or len(filtered_data) == 0:
        print(f"No data found for ImageID: {image_id}")
        continue

    gt_data = gt_data.iloc[0]
    filtered_data = filtered_data.iloc[0]

    # 将结果添加到result_list
    result_list.append({
        'ImageID': image_id,
        'GT_labelID': ','.join(map(str, gt_labels)),
        'Filtered_labelID': ','.join(map(str, filtered_labels)),
        'Result': result,
        'GT_x_center': gt_data['x center'],
        'GT_y_center': gt_data['y center'],
        'GT_width': gt_data['width'],
        'GT_height': gt_data['height'],
        'GT_Area': gt_data['Area'],
        'GT_boxID': gt_data['boxID'],
        'Filtered_x_center': filtered_data['x center'],
        'Filtered_y_center': filtered_data['y center'],
        'Filtered_width': filtered_data['width'],
        'Filtered_height': filtered_data['height'],
        'Filtered_Area': filtered_data['Area'],
    })

# 将结果列表转换为DataFrame
result_df = pd.DataFrame(result_list)

result_df.to_csv('comparison_result.csv', index=False)

No data found for ImageID: 4.0
No data found for ImageID: 8.0
No data found for ImageID: 76.0
No data found for ImageID: 192.0
No data found for ImageID: 294.0
No data found for ImageID: 469.0
No data found for ImageID: 470.0
No data found for ImageID: 665.0
No data found for ImageID: 782.0
No data found for ImageID: 850.0
No data found for ImageID: 982.0
No data found for ImageID: 1127.0
No data found for ImageID: 1215.0
No data found for ImageID: 1472.0
No data found for ImageID: 1889.0
No data found for ImageID: 1891.0
No data found for ImageID: 2034.0
No data found for ImageID: 2119.0
No data found for ImageID: 2125.0
No data found for ImageID: 2180.0
No data found for ImageID: 2201.0
No data found for ImageID: 2644.0
No data found for ImageID: 2751.0
No data found for ImageID: 2965.0
No data found for ImageID: 3339.0
No data found for ImageID: 3359.0
No data found for ImageID: 3367.0
No data found for ImageID: 3371.0
No data found for ImageID: 3378.0


In [4]:
# 读取CSV文件
comparison_df = pd.read_csv('comparison_result.csv')
filtered_df = pd.read_csv('filtered_predTag_1.csv')

# 将comparison_result.csv中的Filtered_labelID进行拆分并展开
comparison_df['Filtered_labelID'] = comparison_df['Filtered_labelID'].astype(str)  # 确保是字符串
comparison_df_expanded = comparison_df.assign(Filtered_labelID=comparison_df['Filtered_labelID'].str.split(',')).explode('Filtered_labelID')
comparison_df_expanded['Filtered_labelID'] = comparison_df_expanded['Filtered_labelID'].str.strip()  # 去除可能的空格

# 先将值转换为浮点数，然后再转换为整数
comparison_df_expanded['Filtered_labelID'] = comparison_df_expanded['Filtered_labelID'].astype(float).astype(int)

# 合并数据，以ImageID和Filtered_labelID为基准
merged_df = pd.merge(
    comparison_df_expanded,
    filtered_df,
    left_on=['ImageID', 'Filtered_labelID'],
    right_on=['ImageID', 'labelID'],
    suffixes=('_comparison', '_filtered')
)

# 保存结果到新的CSV文件
merged_df.to_csv('image_result.csv', index=False)

print("合并完成，结果已保存到merged_result.csv")

合并完成，结果已保存到merged_result.csv


# 分析

In [5]:
# 计算每种情况的发生次数
result_counts = result_df['Result'].value_counts()

# 计算总样本数
total_samples = len(result_df)

# 计算每种情况的发生概率
probabilities = result_counts / total_samples

# 计算每种情况在样本中发生的概率
consistent_NG_rate = probabilities.get('same NG', 0)
consistent_pass_rate = probabilities.get('same pass', 0)
wrong_kill_rate = probabilities.get('wrong kill', 0)
overkill_rate = probabilities.get('overkill', 0)
miss_kill_rate = probabilities.get('miss kill', 0)

# 打印每种情况的发生概率
print("一致NG率:", consistent_NG_rate)
print("一致通过率:", consistent_pass_rate)
print("错杀率:", wrong_kill_rate)
print("过杀率:", overkill_rate)
print("漏杀率:", miss_kill_rate)

# 保存概率结果到新的CSV文件
probabilities_df = probabilities.reset_index()
probabilities_df.columns = ['Result', 'Probability']
probabilities_df.to_csv('probabilities.csv', index=False)

一致NG率: 0.722189349112426
一致通过率: 0.24763313609467455
错杀率: 0.015088757396449704
过杀率: 0.013609467455621301
漏杀率: 0.0014792899408284023
